In [2]:
import pickle
import pandas as pd

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/gnn_input/train_data.pkl', 'rb') as f:
    embeddings, Y, df, X_all, edges_df_list, attributes_df_list = pickle.load(f)
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/gnn_input/val_data.pkl', 'rb') as f:
    val_embeddings, val_Y, val_df, val_X_all, val_edges_df_list, val_attributes_df_list = pickle.load(f)
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/gnn_input/test_data.pkl', 'rb') as f:
    test_embeddings, test_Y, test_df, test_X_all, test_edges_df_list, test_attributes_df_list = pickle.load(f)

Y=Y.argmax(1)
val_Y=val_Y.argmax(1)
test_Y=test_Y.argmax(1)

import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]

for df in attributes_df_list:
    df["Room Type"] = df["Room Type"].apply(lambda x: room_classes_names.index(x))

# Define a function to create a PyTorch Geometric Data object from the room and edge data for each floorplan
def create_data(attributes_df, edges_df):
    edge_index = edges_df[['source', 'target']].values.T
    x = attributes_df[['Relative Area', 'Number of neighboring rooms']].values
    y = attributes_df['Room Type'].values
    return Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=torch.tensor(y, dtype=torch.long))

# Create a list of Data objects, one for each floorplan
data_list = []
for i in range(len(attributes_df_list)):
    attributes_df_i = attributes_df_list[i]
    edges_df_i = edges_df_list[i]
    data_list.append(create_data(attributes_df_i, edges_df_i))

# Define the GCN architecture
class Net(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Create the GCN and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_features=2, num_classes=len(room_classes_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Define the training loop
def train(model, optimizer, data_loader):
    model.train()
    total_loss = 0
    for data in data_loader:
        optimizer.zero_grad()
        out = model(data.x.to(device), data.edge_index.to(device))
        loss = F.nll_loss(out, data.y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(data_loader.dataset)

# Train the GCN on your floorplan data
num_classes = len(room_classes_names)
model = Net(num_features=2, num_classes=len(room_classes_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
data_loader = DataLoader(data_list, batch_size=32, shuffle=True)
for epoch in range(200):
    loss = train(model, optimizer, data_loader)
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss))

# Define a function to create a PyTorch Geometric Data object from the room and edge data for each floorplan
def create_data(attributes_df, edges_df):
    edge_index = edges_df[['source', 'target']].values.T
    x = attributes_df[[ 'Relative Area', 'Number of neighboring rooms']].values
    return Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=None)

# Create a list of Data objects for the test floorplans
test_data_list = []
for i in range(len(test_attributes_df_list)):
    attributes_df_i = test_attributes_df_list[i].drop('Room Type', axis=1)
    edges_df_i = test_edges_df_list[i]
    test_data_list.append(create_data(attributes_df_i, edges_df_i))

# Use the trained model to predict room types for the test data
test_data_loader = DataLoader(test_data_list, batch_size=32, shuffle=False)
model.eval()
predictions = []
with torch.no_grad():
    for data in test_data_loader:
        logits = model(data.x.to(device), data.edge_index.to(device))
        pred = logits.argmax(dim=1)
        pred_room_types = [room_classes_names[i] for i in pred.tolist()]
        # Create a list of None values with the same length as the number of nodes in the graph
        pred_room_types = pred_room_types + [None]*(len(data.x) - len(pred_room_types))
        predictions.extend(pred_room_types)

# Create a list of actual room types for the test data
actual_room_types = []
for attributes_df_i in test_attributes_df_list:
    actual_room_types.extend(attributes_df_i['Room Type'].tolist())

# Compare the actual and predicted room types and calculate accuracy
correct_predictions = 0
for i in range(len(actual_room_types)):
    if actual_room_types[i] == predictions[i]:
        correct_predictions += 1
accuracy = correct_predictions / len(actual_room_types)

print('Accuracy: {:.2f}%'.format(accuracy * 100))

/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 000, Loss: 2.47386
Epoch: 001, Loss: 2.21874
Epoch: 002, Loss: 2.05633
Epoch: 003, Loss: 1.98787
Epoch: 004, Loss: 1.91551
Epoch: 005, Loss: 1.88430
Epoch: 006, Loss: 1.84578
Epoch: 007, Loss: 1.82427
Epoch: 008, Loss: 1.80134
Epoch: 009, Loss: 1.80292
Epoch: 010, Loss: 1.77379
Epoch: 011, Loss: 1.76732
Epoch: 012, Loss: 1.75969
Epoch: 013, Loss: 1.75727
Epoch: 014, Loss: 1.74891
Epoch: 015, Loss: 1.73789
Epoch: 016, Loss: 1.74057
Epoch: 017, Loss: 1.73259
Epoch: 018, Loss: 1.71755
Epoch: 019, Loss: 1.71741
Epoch: 020, Loss: 1.72106
Epoch: 021, Loss: 1.71523
Epoch: 022, Loss: 1.71198
Epoch: 023, Loss: 1.70548
Epoch: 024, Loss: 1.69579
Epoch: 025, Loss: 1.69044
Epoch: 026, Loss: 1.69541
Epoch: 027, Loss: 1.68532
Epoch: 028, Loss: 1.68369
Epoch: 029, Loss: 1.66691
Epoch: 030, Loss: 1.66656
Epoch: 031, Loss: 1.67537
Epoch: 032, Loss: 1.65016
Epoch: 033, Loss: 1.65119
Epoch: 034, Loss: 1.63861
Epoch: 035, Loss: 1.63700
Epoch: 036, Loss: 1.64304
Epoch: 037, Loss: 1.62820
Epoch: 038, 

/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [3]:
from sklearn.metrics import classification_report

# Assume actual_room_types and predictions are defined

print('Accuracy: {:.2f}%'.format(accuracy * 100))

report = classification_report(actual_room_types, predictions)
print(report)


Accuracy: 40.74%
              precision    recall  f1-score   support

        Bath       0.47      0.51      0.49        49
    Bed Room       0.36      0.23      0.28        53
       Entry       0.44      0.67      0.53        43
     Kitchen       0.31      0.34      0.33        41
 Living Room       0.41      0.45      0.43        42
     Storage       0.00      0.00      0.00        15

    accuracy                           0.41       243
   macro avg       0.33      0.37      0.34       243
weighted avg       0.38      0.41      0.38       243



/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [4]:
# Save the trained model to a file
torch.save(model.state_dict(), 'model.pt')

In [5]:
# Load the saved model from file
model_state_dict = torch.load('model.pt')

# Re-create the model using the same architecture definition as before
model = Net(num_features=2, num_classes=len(room_classes_names)).to(device)

# Load the saved model parameters into the re-created model
model.load_state_dict(model_state_dict)

<All keys matched successfully>